# learner

In [1]:
#| default_exp learner

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export
from enum import Enum
from tqdm.auto import tqdm
from scipy import sparse
from packaging import version
import torch, re, math, numpy as np, os, time, datasets, pickle, transformers
from typing import Any, Tuple, Optional, Sequence, Union, Dict, List, NamedTuple
from transformers import AutoTokenizer, BatchEncoding, Seq2SeqTrainer, Seq2SeqTrainingArguments

import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler

from torch.nn.parallel import DataParallel
from torch.nn.parallel._functions import Scatter
from torch.nn.parallel.scatter_gather import _is_namedtuple

from xcai.core import *
from xcai.data import *
from xcai.sdata import *
from xcai.representation.search import *
from xcai.generation.trie import *
from xcai.generation.generate import *
from xcai.clustering.cluster import *
from xcai.transform import PadFeatTfm
from xcai.optimizers.oakX import MultipleOptimizer, MultipleScheduler

from fastcore.utils import *
from fastcore.meta import *

In [4]:
#| export
from transformers.trainer_pt_utils import (
    find_batch_size, 
    nested_concat, nested_numpify, 
    IterableDatasetShard, 
    get_dataloader_sampler, 
    get_model_param_count,
    LengthGroupedSampler
)
from transformers.trainer_utils import has_length, denumpify_detensorize, speed_metrics, TrainOutput, HPSearchBackend, seed_worker
from transformers.trainer_callback import TrainerState, ExportableState
from transformers.trainer import _is_peft_model
from transformers.modeling_utils import unwrap_model
from transformers.utils import is_sagemaker_mp_enabled, is_accelerate_available, is_torch_xla_available, logging, is_datasets_available
from transformers.debug_utils import DebugOption, DebugUnderflowOverflow

from transformers.integrations import hp_params
from transformers.integrations.tpu import tpu_spmd_dataloader
from transformers.integrations.deepspeed import deepspeed_init, deepspeed_load_checkpoint, is_deepspeed_available
from transformers.trainer_utils import RemoveColumnsCollator

if is_accelerate_available():
    from accelerate import Accelerator, skip_first_batches
    from accelerate import __version__ as accelerate_version
    from accelerate.utils import (
        DistributedDataParallelKwargs,
        DistributedType,
        GradientAccumulationPlugin,
        load_fsdp_model,
        load_fsdp_optimizer,
        save_fsdp_model,
        save_fsdp_optimizer,
    )

    DATA_SAMPLERS = [RandomSampler]
    if version.parse(accelerate_version) > version.parse("0.23.0"):
        from accelerate.data_loader import SeedableRandomSampler

        DATA_SAMPLERS += [SeedableRandomSampler]

    if is_deepspeed_available():
        from accelerate.utils import DeepSpeedSchedulerWrapper

if is_accelerate_available("0.28.0"):
    from accelerate.utils import DataLoaderConfiguration

TRAINING_ARGS_NAME = "training_args.bin"
TRAINER_STATE_NAME = "trainer_state.json"
OPTIMIZER_NAME = "optimizer.pt"
OPTIMIZER_NAME_BIN = "optimizer.bin"
SCHEDULER_NAME = "scheduler.pt"
SCALER_NAME = "scaler.pt"
FSDP_MODEL_NAME = "pytorch_model_fsdp"

logger = logging.get_logger(__name__)

In [5]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [6]:
#| hide
from xcai.block import *
from xcai.models.PPP0XX import *
from xcai.metrics import *

In [7]:
#| export
from xcai.data import *

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

## Setup

In [ ]:
#| hide
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'train')

In [ ]:
#| hide
batch = block.train.one_batch(11)

In [ ]:
#| hide
m = BT0002.from_pretrained('bert-base-uncased', tn_targ=10_000, ig_tok=0)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0002 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#| hide
batch.keys()

dict_keys(['lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask'])

In [ ]:
#| hide
b = prepare_batch(m, batch, m_args='lbl2data_idx')

In [ ]:
#| hide
b.keys()

dict_keys(['lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask'])

In [ ]:
#| hide
m = m.to('cuda')
b = b.to('cuda')

In [ ]:
#| hide
o = m(**b)

In [ ]:
#| hide
o.loss

tensor(14.9452, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/'
pkl_file = f'{pkl_dir}/processed/wikiseealsotitles_data-cat-lnk_distilbert-base-uncased_xcs.pkl'

In [ ]:
with open(pkl_file, 'rb') as file: block = pickle.load(file)

## DataParallel

In [8]:
#| export
def scatter(inputs, target_gpus, chunk_sizes=None, dim=0):
    def scatter_map(obj):
        if isinstance(obj, torch.Tensor):
            return Scatter.apply(target_gpus, chunk_sizes, dim, obj)
        if _is_namedtuple(obj):
            return [type(obj)(*args) for args in zip(*map(scatter_map, obj))]
        if isinstance(obj, tuple) and len(obj) > 0:
            return list(zip(*map(scatter_map, obj)))
        if isinstance(obj, list) and len(obj) > 0:
            return [list(i) for i in zip(*map(scatter_map, obj))]
        if isinstance(obj, dict) and len(obj) > 0:
            return [type(obj)(i) for i in zip(*map(scatter_map, obj.items()))]
        return [obj for _ in target_gpus] 
    try:
        res = scatter_map(inputs)
    finally:
        scatter_map = None
    return res
    
def scatter_kwargs(
    inputs: Tuple[Any, ...],
    kwargs: Optional[Dict[str, Any]],
    target_gpus: Sequence[Union[int, torch.device]],
    chunk_sizes: Optional[Sequence[int]]=None,
    dim: int = 0,
) -> Tuple[Tuple[Any, ...], Tuple[Dict[str, Any], ...]]:
    scattered_inputs = scatter(inputs, target_gpus, chunk_sizes, dim) if inputs else []
    scattered_kwargs = scatter(kwargs, target_gpus, chunk_sizes, dim) if kwargs else []
    if len(scattered_inputs) < len(scattered_kwargs):
        scattered_inputs.extend(() for _ in range(len(scattered_kwargs) - len(scattered_inputs)))
    elif len(scattered_kwargs) < len(inputs):
        scattered_kwargs.extend({} for _ in range(len(scattered_inputs) - len(scattered_kwargs)))
    return scattered_inputs, scattered_kwargs
    

In [9]:
#| export
class XCDataParallel(DataParallel):

    @delegates(DataParallel.__init__)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def _get_feat_name(self, x:Optional[Dict[str, Any]]):
        return list(set([k.split('_', maxsplit=1)[0] for k in x]))
    
    def _extract_feat(self, x:Optional[Dict[str, Any]], prefix:str):
        return {k:v for k,v in x.items() if re.match(f'^{prefix}_(?!.*2ptr)', k) or re.match(f'^.*_{prefix}2ptr$', k)}

    def scatter(
        self,
        inputs: Tuple[Any, ...],
        kwargs: Optional[Dict[str, Any]],
        device_ids: Sequence[Union[int, torch.device]],
    ) ->Any:
        if len(inputs): raise ValueError('`inputs` should be empty.')    
        feat_name = self._get_feat_name(kwargs)
        
        data_feat = self._extract_feat(kwargs, 'data')
        scattered_inputs, scattered_kwargs = scatter_kwargs(inputs, data_feat, device_ids, None, dim=self.dim)
        feat_name.remove('data')
        
        for k in feat_name:
            ptr_name = f'{k}_data2ptr'
            if ptr_name in scattered_kwargs[0] and scattered_kwargs[0][ptr_name] is not None:
                chunk_sz = [o[ptr_name].sum().item() for o in scattered_kwargs]
                if len(chunk_sz) < len(device_ids): 
                    chunk_sz.extend([0 for _ in range(len(device_ids) - len(chunk_sz))])
                
                feat = self._extract_feat(kwargs, k)
                _, o = scatter_kwargs(inputs, feat, device_ids, chunk_sz, dim=self.dim)
                for p,q in zip(scattered_kwargs, o): p.update(q)
                    
        return tuple(scattered_inputs), tuple(scattered_kwargs)
        

### Example

In [ ]:
import pickle
from transformers import AutoTokenizer, BatchEncoding

tokz = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
data_dir = '/home/aiscuser/scratch/datasets'
pkl_dir = f'{data_dir}/processed/'

In [ ]:
with open(f'{pkl_dir}/wikiseealso_data-metas_distilbert-base-uncased_rm_radga.pkl', 'rb') as file: 
    block = pickle.load(file)

In [ ]:
with open(f'{pkl_dir}/wikiseealso_data-metas_distilbert-base-uncased_xcnlg_radga.pkl', 'rb') as file: 
    block = pickle.load(file)

In [ ]:
b = block.train.one_batch(4)
bb = BatchEncoding({k:v for k,v in b.items() if isinstance(v, torch.Tensor)})

In [ ]:
for k,v in bb.items():
    if 'ptr' in k: print(k, ': ', v.shape)

plbl2data_data2ptr :  torch.Size([4])
lbl2data_data2ptr :  torch.Size([4])
pcat2data_data2ptr :  torch.Size([4])
cat2data_data2ptr :  torch.Size([4])
pcat2lbl2data_data2ptr :  torch.Size([4])
cat2lbl2data_data2ptr :  torch.Size([4])
phlk2data_data2ptr :  torch.Size([4])
hlk2data_data2ptr :  torch.Size([4])
hlk2lbl2data_data2ptr :  torch.Size([4])
hlk2lbl2data_plbl2data2ptr :  torch.Size([6])


In [ ]:
#| hide
class MyModel(nn.Module):

    def forward(self, **kwargs):
        for k,v in kwargs.items(): 
            if isinstance(v, torch.Tensor): print(k, ': ', v, ', ', v.device)
        return kwargs
        

In [ ]:
#| hide
m = XCDataParallel(module=MyModel())

In [ ]:
#| hide
o = m(**bb)

plbl2data_data2ptrplbl2data_data2ptr :   :  tensor([2, 1], device='cuda:0')tensor([1, 2], device='cuda:1')  , ,   cuda:0cuda:1

lbl2data_data2ptrlbl2data_data2ptr  : :   tensor([1, 2], device='cuda:1')tensor([2, 1], device='cuda:0') ,   , cuda:1 
cuda:0pcat2data_data2ptr
 pcat2data_data2ptr:   : tensor([14,  6], device='cuda:1')  tensor([13,  6], device='cuda:0'),   cuda:1,  
cat2data_data2ptrcuda:0
 : cat2data_data2ptr  :  tensor([1, 1], device='cuda:1') tensor([1, 1], device='cuda:0') , ,   cuda:0cuda:1

pcat2lbl2data_data2ptrpcat2lbl2data_data2ptr :  :   tensor([4, 7], device='cuda:1')tensor([0, 4], device='cuda:0') ,   , cuda:0 
cat2lbl2data_data2ptrcuda:1
 cat2lbl2data_data2ptr:   :  tensor([0, 1], device='cuda:0') tensor([1, 1], device='cuda:1'),   , cuda:0 
cuda:1phlk2data_data2ptr
 phlk2data_data2ptr:   : tensor([16, 18], device='cuda:0')  , tensor([15, 40], device='cuda:1') cuda:0 
, hlk2data_data2ptr  cuda:1: 
hlk2data_data2ptr  tensor([3, 3], device='cuda:0'):   , tensor([3,

In [ ]:
np.all([torch.all(bb[k] == o[k].to('cpu')) for k in o.keys()])

True

## Learner

In [10]:
#| export
@patch
def __call__(self:RemoveColumnsCollator, features):
    if isinstance(features, list):
        features = [self._remove_columns(feature) for feature in features]
    elif isinstance(features, dict) or isinstance(features, BatchEncoding):
        features = self._remove_columns(features)
    else:
        raise ValueError(f'Invalid input type: {type(features)}')
    return self.data_collator(features)

In [11]:
#| export
class XCEvalLoopOutput(NamedTuple):
    pred_idx: Union[np.ndarray, Tuple[np.ndarray]]
    pred_ptr: Union[np.ndarray, Tuple[np.ndarray]]
    pred_score: Union[np.ndarray, Tuple[np.ndarray]]
    targ_idx: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    targ_ptr: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    gen_output: Optional[Dict]
    repr_output: Optional[Dict]
    metrics: Optional[Dict[str, float]]
    num_samples: Optional[int]

class XCPredictionOutput(NamedTuple):
    pred_idx: Union[np.ndarray, Tuple[np.ndarray]]
    pred_ptr: Union[np.ndarray, Tuple[np.ndarray]]
    pred_score: Optional[Union[np.ndarray, Tuple[np.ndarray]]]
    gen_output: Optional[Dict]
    repr_output: Optional[Dict]
    metrics: Optional[Dict[str, float]]
    num_samples: Optional[int]
    

In [12]:
#| export
class ParallelMode(Enum):
    NOT_PARALLEL = "not_parallel"
    NOT_DISTRIBUTED = "not_distributed"
    DISTRIBUTED = "distributed"
    SAGEMAKER_MODEL_PARALLEL = "sagemaker_model_parallel"
    SAGEMAKER_DATA_PARALLEL = "sagemaker_data_parallel"
    TPU = "tpu"

In [13]:
#| export
class XCLearningArguments(Seq2SeqTrainingArguments):

    @delegates(Seq2SeqTrainingArguments.__init__)
    def __init__(self, 
                 use_encoder_parallel:Optional[bool]=False,
                 generation_length_penalty:Optional[float]=1.0,
                 generation_eos_token:Optional[int]=102,
                 generation_num_beams:Optional[int]=5,
                 generation_max_info:Optional[int]=None,
                 representation_accumulation_steps:Optional[int]=None,
                 
                 output_representation_attribute:Optional[str]='data_repr',
                 label_representation_attribute:Optional[str]='data_repr',
                 representation_attribute:Optional[str]='data_repr',

                 use_data_metadata_for_representation:Optional[bool]=False,
                 
                 representation_num_beams:Optional[int]=5,
                 representation_search_type:Optional[str]='INDEX',
                 index_space:Optional[str]='cosine', 
                 index_efc:Optional[int]=300, 
                 index_m:Optional[int]=100, 
                 index_efs:Optional[int]=300,
                 index_num_threads:Optional[int]=84,
                 use_cpu_for_searching:Optional[bool]=False,
                 predict_with_generation:Optional[bool]=False,
                 predict_with_representation:Optional[bool]=False,
                 output_concatenation_weight:Optional[float]=1.0,
                 group_by_cluster:Optional[bool]=False,
                 num_clustering_warmup_epochs:Optional[int]=None,
                 num_cluster_update_epochs:Optional[int]=1,
                 num_cluster_size_update_epochs:Optional[int]=1,
                 clustering_type:Optional[str]='EXPO',
                 minimum_clusters:Optional[int]=3,
                 maximum_clusters:Optional[int]=None,
                 minimum_cluster_size:Optional[int]=1,
                 maximum_cluster_size:Optional[int]=None,
                 clustering_devices:Optional[List]=None,
                 use_cpu_for_clustering:Optional[bool]=True,
                 use_data_metadata_for_clustering:Optional[bool]=False,
                 clustering_representation_attribute:Optional[str]='data_repr',
                 
                 target_indices_key:Optional[str]='lbl2data_idx',
                 target_pointer_key:Optional[str]='lbl2data_data2ptr',
                 
                 data_aug_meta_name:Optional[str]=None,
                 data_aug_prefix:Optional[str]=None,
                 augmentation_num_beams:Optional[int]=3,
                 predict_with_augmentation:Optional[bool]=False,
                 use_augmentation_index_representation:Optional[bool]=False,
                 metadata_representation_attribute:Optional[str]='data_repr',
                 data_augmentation_attribute:Optional[str]='data_repr',
                 data_meta_batch_size:Optional[int]=2048,

                 augment_metadata:Optional[bool]=False,
                 num_metadata_augment_epochs:Optional[int]=1,
                 num_metadata_augment_warmup_epochs:Optional[int]=10,

                 centroid_data_batch_size:Optional[int]=2048,
                 centroid_data_attribute_representation:Optional[str]='data_repr',
                 use_centroid_data_metadata:Optional[bool]=False,
                 use_centroid_label_representation:Optional[bool]=False,

                 use_teacher_lbl_representation:Optional[bool]=False,
                 use_teacher_data_representation:Optional[bool]=False,
                 
                 use_distributional_representation:Optional[bool]=False,
                 use_label_metadata:Optional[bool]=True,
                 prune_metadata:Optional[bool]=False,
                 num_metadata_prune_epochs:Optional[int]=1,
                 metadata_prune_batch_size:Optional[int]=64,
                 num_metadata_prune_warmup_epochs:Optional[int]=10,
                 prune_metadata_names:Optional[List]=None,
                 use_data_metadata_for_pruning:Optional[bool]=True,
                 prune_metadata_threshold:Optional[float]=0.0,
                 prune_metadata_topk:Optional[int]=None,

                 free_parameter_warmup_steps:Optional[int]=0,
                 free_parameter_lr_coefficient:[float]=1.0,

                 mix_metadata:Optional[bool]=False,
                 num_mix_metadata_epochs:Optional[int]=5,
                 num_mix_metadata_warmup_epochs:Optional[int]=10,
                 maximum_mix_metadata_epochs:Optional[int]=50,
                 mix_metadata_name_1:Optional[str]=None,
                 mix_metadata_name_2:Optional[str]=None,
                 mix_metadata_k:Optional[int]=3,

                 use_memory_aggressively:Optional[bool]=False,
                 
                 **kwargs):
        super().__init__(**kwargs)
        store_attr('output_representation_attribute,representation_attribute,label_representation_attribute')
        store_attr('clustering_representation_attribute,use_data_metadata_for_clustering')
        
        store_attr('generation_num_beams,generation_length_penalty,generation_max_info,generation_eos_token')
        store_attr('representation_accumulation_steps,representation_num_beams,representation_search_type')
        store_attr('index_space,index_efc,index_m,index_efs,index_num_threads,use_cpu_for_searching')
        store_attr('predict_with_generation,predict_with_representation,output_concatenation_weight')
        store_attr('group_by_cluster,num_cluster_update_epochs,num_cluster_size_update_epochs,num_clustering_warmup_epochs')
        store_attr('clustering_devices,clustering_type,maximum_cluster_size,use_cpu_for_clustering')
        store_attr('target_indices_key,target_pointer_key')
        store_attr('use_encoder_parallel')
        store_attr('data_aug_meta_name,data_aug_prefix,augmentation_num_beams,predict_with_augmentation')
        store_attr('use_augmentation_index_representation,metadata_representation_attribute,data_augmentation_attribute')
        store_attr('use_distributional_representation')
        store_attr('use_data_metadata_for_representation')
        
        store_attr('use_label_metadata,data_meta_batch_size,augment_metadata,num_metadata_augment_epochs,num_metadata_augment_warmup_epochs')
        store_attr('use_centroid_data_metadata,use_centroid_label_representation,centroid_data_attribute_representation,centroid_data_batch_size')
        store_attr('use_teacher_lbl_representation')
        store_attr('use_teacher_data_representation')
        
        store_attr('prune_metadata,num_metadata_prune_epochs,num_metadata_prune_warmup_epochs,metadata_prune_batch_size,prune_metadata_names')
        store_attr('use_data_metadata_for_pruning,prune_metadata_threshold,prune_metadata_topk')

        store_attr('free_parameter_warmup_steps,free_parameter_lr_coefficient')

        store_attr('mix_metadata,num_mix_metadata_epochs,num_mix_metadata_warmup_epochs,maximum_mix_metadata_epochs')
        store_attr('mix_metadata_name_1,mix_metadata_name_2,mix_metadata_k')

        store_attr('use_memory_aggressively')
        self.minimum_clusters = max(1, minimum_clusters)
        self.maximum_clusters = max(minimum_clusters, maximum_clusters) if maximum_clusters is not None else minimum_clusters
        self.minimum_cluster_size = max(1, minimum_cluster_size)
        
        self.maximum_mix_metadata_epochs = min(maximum_mix_metadata_epochs, self.num_train_epochs)
        

### `XCLearner`

In [14]:
#| export
class XCLearner(Seq2SeqTrainer):

    @delegates(Seq2SeqTrainer.__init__)
    def __init__(self, 
                 trie:Optional[Trie]=None, 
                 **kwargs):
        super().__init__(**kwargs)
        self.tbs = TrieBeamSearch(trie, self.args.generation_eos_token, n_bm=self.args.generation_num_beams, 
                                  len_penalty=self.args.generation_length_penalty, max_info=self.args.generation_max_info, **kwargs)
        self.idxs = (
            BruteForceSearch(n_bm=self.args.representation_num_beams)
            if self.args.representation_search_type == 'BRUTEFORCE' else
            IndexSearch(space=self.args.index_space, efc=self.args.index_efc, m=self.args.index_m, 
                        efs=self.args.index_efs, n_bm=self.args.representation_num_beams, 
                        n_threads=self.args.index_num_threads) 
        )
        self.aug_idxs = None

    def _wrap_model(self, model, training=True, dataloader=None):
        if unwrap_model(model) is not model:
            return model

        if self.args.n_gpu > 1:
            if (hasattr(model, 'encoder') and isinstance(model.encoder, nn.DataParallel)) or self.args.use_encoder_parallel: return model
            else: return XCDataParallel(module=model)
        return model

    def evaluate(self, eval_dataset:Optional[Dataset]=None, ignore_keys:Optional[List[str]]=None, 
             metric_key_prefix:str="eval", **gen_kwargs):
        gen_kwargs = gen_kwargs.copy()
        if gen_kwargs.get("length_penalty") is None and self.args.generation_length_penalty is not None:
            gen_kwargs["length_penalty"] = self.args.generation_length_penalty
        if gen_kwargs.get("gen_num_beams") is None and self.args.generation_num_beams is not None:
            gen_kwargs["gen_num_beams"] = self.args.generation_num_beams
        if gen_kwargs.get("repr_num_beams") is None and self.args.representation_num_beams is not None:
            gen_kwargs["repr_num_beams"] = self.args.representation_num_beams
        if gen_kwargs.get("aug_num_beams") is None and self.args.augmentation_num_beams is not None:
            gen_kwargs["aug_num_beams"] = self.args.augmentation_num_beams
            
        self.gather_function, self._gen_kwargs  = self.accelerator.gather, gen_kwargs
        
        return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)

    def predict(self, test_dataset: Dataset, ignore_keys:Optional[List[str]]=None, 
            metric_key_prefix:str="test", **gen_kwargs):
        gen_kwargs = gen_kwargs.copy()
        if gen_kwargs.get("length_penalty") is None and self.args.generation_length_penalty is not None:
            gen_kwargs["length_penalty"] = self.args.generation_length_penalty
        if gen_kwargs.get("gen_num_beams") is None and self.args.generation_num_beams is not None:
            gen_kwargs["gen_num_beams"] = self.args.generation_num_beams
        if gen_kwargs.get("repr_num_beams") is None and self.args.representation_num_beams is not None:
            gen_kwargs["repr_num_beams"] = self.args.representation_num_beams
        if gen_kwargs.get("aug_num_beams") is None and self.args.augmentation_num_beams is not None:
            gen_kwargs["aug_num_beams"] = self.args.augmentation_num_beams
    
        self.gather_function, self._gen_kwargs = self.accelerator.gather, gen_kwargs
        self._memory_tracker.start()
    
        test_dataloader = self.get_test_dataloader(test_dataset)
        start_time = time.time()
    
        output = self.evaluation_loop(test_dataloader, description="Prediction", ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(metric_key_prefix,start_time,num_samples=output.num_samples,num_steps=math.ceil(output.num_samples / total_batch_size),)
        )
        self.control = self.callback_handler.on_predict(self.args, self.state, self.control, output.metrics)
        self._memory_tracker.stop_and_update_metrics(output.metrics)
        return XCPredictionOutput(pred_idx=output.pred_idx, pred_ptr=output.pred_ptr, pred_score=output.pred_score, 
                              gen_output=output.gen_output, repr_output=output.repr_output, metrics=output.metrics, 
                              num_samples=output.num_samples)
    
    def _gather_host_output(self, output, host_output):
        if output is not None:
            output = self.accelerator.pad_across_processes(output, dim=1, pad_index=-100)
            output = self.gather_function((output))
            return output if host_output is None else nested_concat(host_output, output, padding_index=-100)
        else: return host_output

    def _gather_all_output(self, host_output, all_output, to_cpu=True):
        if host_output is not None:
            if isinstance(host_output, torch.Tensor) and to_cpu: host_output = host_output.cpu()
            return host_output if all_output is None else nested_concat(all_output, host_output, padding_index=-100)
        else: return all_output

    def create_optimizer_and_scheduler(self, num_training_steps: int):
        if hasattr(self.model, 'meta_embeddings') and self.model.meta_embeddings.sparse:
            NO_DECAY = ['bias', 'LayerNorm.weight']
        
            dense, sparse = [], []
            for k, p in self.model.named_parameters():
                if p.requires_grad:
                    if "meta_embeddings" not in k: dense.append((k,p))
                    else: sparse.append(p)
        
            params = [
                {'params': [p for n, p in dense if not any(nd in n for nd in NO_DECAY)], 'weight_decay': 0.01},
                {'params': [p for n, p in dense if any(nd in n for nd in NO_DECAY)], 'weight_decay': 0.0},
            ]
        
            optimizer_list = [torch.optim.AdamW(params, **{'lr': self.args.learning_rate, 'eps': 1e-6}),
                              torch.optim.SparseAdam(sparse, **{'lr': self.args.learning_rate * self.args.free_parameter_lr_coefficient, 'eps': 1e-6})]
        
            self.optimizer = MultipleOptimizer(optimizer_list)
            scheduler_list = [transformers.get_linear_schedule_with_warmup(self.optimizer.optimizers[0], num_warmup_steps=self.args.warmup_steps,
                                                                           num_training_steps=num_training_steps),
                                transformers.get_cosine_schedule_with_warmup(self.optimizer.optimizers[1],
                                                                             num_warmup_steps=self.args.free_parameter_warmup_steps,
                                                                             num_training_steps=num_training_steps)]
        
            self.lr_scheduler = MultipleScheduler(scheduler_list)
        else:
            super().create_optimizer_and_scheduler(num_training_steps)
            

In [15]:
#| export
@patch
def _get_dataset(self:XCLearner, dataset:Dataset, dset_type:str='lbl', use_metadata:Optional[bool]=False):
    dset = get_attr(dataset, f'{dset_type}_dset')

    data_aug_prefix = self.args.data_aug_meta_name if self.args.data_aug_prefix is None else self.args.data_aug_prefix
    meta_name = f'{data_aug_prefix}_meta' if data_aug_prefix is not None else None
    
    if (
        meta_name is not None and 
        dataset.meta is not None and 
        meta_name in dataset.meta and 
        use_metadata
    ):    
        meta_dset= dataset.meta[meta_name]
        prefix,meta,meta_info  = meta_dset.prefix, get_attr(meta_dset, f'{dset_type}_meta'), meta_dset.meta_info

        if dset_type == 'data':
            metadata = type(meta_dset)(prefix=prefix, data_meta=meta, lbl_meta=sparse.csr_matrix((dataset.n_lbl, meta_dset.n_meta)), meta_info=meta_info)            
        elif dset_type == 'lbl':
            metadata = type(meta_dset)(prefix=prefix, data_meta=meta, lbl_meta=sparse.csr_matrix((dataset.n_data, meta_dset.n_meta)), meta_info=meta_info)
        else: raise ValueError(f'Invalid `dset_type`, should be one of ["data", "lbl"].')

        if isinstance(metadata, SMetaXCDataset):
            metadata.n_data_meta_samples = meta_dset.n_data_meta_samples
            metadata.n_sdata_meta_samples = meta_dset.n_sdata_meta_samples
            metadata.meta_oversample = meta_dset.meta_oversample
        elif isinstance(metadata, MetaXCDataset):
            metadata.n_data_meta_samples = self.args.augmentation_num_beams
        else: raise ValueError('Invalid meta dataset.')

        meta_kwargs = {meta_name: metadata}
        dset = type(dataset)(dset, **meta_kwargs)
        
    return dset


In [16]:
#| export
@patch
def _build_aug_index(self:XCLearner, dataset:Optional[Dataset]=None):
    dataset = dataset if self.eval_dataset is None else self.eval_dataset
    dataset = dataset if self.train_dataset is None else self.train_dataset
    
    meta_name = f'{self.args.data_aug_meta_name}_meta' if self.args.data_aug_meta_name is not None else None
    if (
        dataset is not None and 
        dataset.meta is not None and 
        meta_name is not None and 
        meta_name in dataset.meta
    ):
        self.aug_idxs = IndexSearch(space=self.args.index_space, efc=self.args.index_efc, m=self.args.index_m, 
                                    efs=self.args.index_efs, n_bm=self.args.augmentation_num_beams, 
                                    n_threads=self.args.index_num_threads)
        
        dset = type(dataset.data)(data_info=getattr(dataset.meta[meta_name], 'meta_info'))
        dataloader = self.get_test_dataloader(dset)
        rep = self.get_meta_representation(dataloader, to_cpu=isinstance(self.aug_idxs, IndexSearch))
        self.aug_idxs.build(rep)

@patch
def _build_lbl_index(self:XCLearner, dataset:Optional[Dataset]=None):
    dataset = self.eval_dataset if dataset is None or dataset.lbl_info is None else dataset
    dataset = self.train_dataset if dataset is None or dataset.lbl_info is None else dataset
    
    if dataset is not None: 
        to_cpu = True if isinstance(self.idxs, IndexSearch) else self.args.use_cpu_for_searching
        lbl_rep = self._get_lbl_representation(dataset, to_cpu=to_cpu)
        self.idxs.build(lbl_rep)
    else: raise ValueError('Failed to build `self.idxs`')
        

In [17]:
#| export
@patch
def _get_lbl_representation(self:XCLearner, dataset:Optional[Dataset]=None, to_cpu:Optional[bool]=False):
    if dataset is not None:
        if self.args.use_centroid_label_representation:
            dataset = dataset if self.train_dataset else self.train_dataset
            
            if self.args.use_teacher_data_representation:
                if not hasattr(self.model, 'm_teacher'): raise ValueError('Model does not contain `m_teacher`.')
                data_rep = self.model.m_teacher.get_data_embeddings().data.cpu()
            else:
                dset = self._get_dataset(dataset, dset_type='data', use_metadata=self.args.use_centroid_data_metadata)
                dataloader = self.get_test_dataloader(dset)
                data_rep = self.get_representation(dataloader, representation_attribute=self.args.centroid_data_attribute_representation)
            
            lbl_data = dataset.data.data_lbl.T.tocsr()
            
            lbl_rep = None
            for i in tqdm(range(0, lbl_data.shape[0], self.args.centroid_data_batch_size)):
                data = lbl_data[i:i+self.args.centroid_data_batch_size]
                rep = data@data_rep
                rep = rep/data.getnnz(axis=1).reshape(-1, 1)
                rep = (
                    torch.tensor(rep, dtype=data_rep.dtype) 
                    if isinstance(self.idxs, IndexSearch) else 
                    torch.tensor(rep, device=self.model.device, dtype=data_rep.dtype)
                )
                lbl_rep = rep if lbl_rep is None else torch.cat([lbl_rep,rep], dim=0)
                
        elif self.args.use_teacher_lbl_representation:
            if not hasattr(self.model, 'm_teacher'): raise ValueError('Model does not contain `m_teacher`.')
            lbl_rep = self.model.m_teacher.get_lbl_embeddings().data
        else:
            dset = self._get_dataset(dataset, dset_type='lbl', use_metadata=self.args.use_label_metadata)
            dataloader = self.get_test_dataloader(dset)
            
            if hasattr(self.model, 'get_label_representation'): lbl_rep = self.get_label_representation(dataloader, to_cpu=to_cpu)
            else:lbl_rep = self.get_representation(dataloader, representation_attribute=self.args.label_representation_attribute, to_cpu=to_cpu)
                
        return lbl_rep
    else: 
        raise ValueError('`dataset` is None, could not create label representation.')
        

In [18]:
#| export
@patch
def get_label_representation(self:XCLearner, dataloader: DataLoader, to_cpu:Optional[bool]=True):
    data_host, all_data = None, None
    
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        use_noise = self.model.disable_noise()
    
    for step, inputs in tqdm(enumerate(dataloader), total=len(dataloader)):
        inputs = inputs.to(self.model.device)
        with torch.no_grad(): data = getattr(self.model.get_label_representation(**inputs), self.args.label_representation_attribute)
        data_host = self._gather_host_output(data, data_host)
        if self.args.representation_accumulation_steps is not None and (step + 1) % self.args.representation_accumulation_steps == 0:
            all_data, data_host = self._gather_all_output(data_host, all_data, to_cpu=to_cpu), None
            
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        self.model.set_noise(use_noise)
            
    return self._gather_all_output(data_host, all_data, to_cpu=to_cpu)
    

In [19]:
#| export
@patch
def generation_output(
    self:XCLearner,
    model:nn.Module,
    inputs:Dict[str, Union[torch.Tensor, Any]],
    **kwargs
):
    inputs = self._prepare_inputs(inputs)
    n_bm = kwargs.pop("gen_num_beams") if "gen_num_beams" in kwargs and kwargs["gen_num_beams"] is not None else self.args.generation_num_beams
    len_penalty = kwargs.pop("length_penalty") if "length_penalty" in kwargs and kwargs["length_penalty"] is not None else self.args.generation_length_penalty
    
    with torch.no_grad(): o = self.tbs.proc(self.model, inputs.copy(), n_bm=n_bm, len_penalty=len_penalty)
        
    return {'pred_idx':o['info2seq2data_idx'], 'pred_score':o['info2seq2data_score'], 'pred_ptr':o['info2seq2data_data2ptr']}

@patch
def representation_output(
    self:XCLearner,
    model:nn.Module,
    inputs:Dict[str, Union[torch.Tensor, Any]],
    **kwargs
):
    inputs = self._prepare_inputs(inputs)
    n_bm = kwargs.pop("repr_num_beams") if "repr_num_beams" in kwargs and kwargs["repr_num_beams"] is not None else self.args.representation_num_beams
    
    with torch.no_grad(): 
        o = model(**inputs)
        if getattr(o, self.args.output_representation_attribute) is not None:
            o = getattr(o, self.args.output_representation_attribute)
        else:
            o = getattr(o, self.args.representation_attribute)
        if self.args.use_distributional_representation: o = o.exp()
            
    o = self.idxs.proc(o, n_bm=n_bm)
        
    return {'pred_idx':o['info2data_idx'], 'pred_score':o['info2data_score'], 'pred_ptr':o['info2data_data2ptr']}

@patch
def augmentation_output(
    self:XCLearner,
    model:nn.Module,
    inputs:Dict[str, Union[torch.Tensor, Any]],
    **kwargs
):
    if self.aug_idxs is None: 
        raise ValueError('Augmentation index `aug_idx` is not initialized.')
        
    inputs = self._prepare_inputs(inputs)
    n_bm = kwargs.pop("aug_num_beams") if "aug_num_beams" in kwargs and kwargs["aug_num_beams"] is not None else self.args.augmentation_num_beams
    
    with torch.no_grad(): 
        inputs = {'data_input_ids':inputs['data_input_ids'], 'data_attention_mask':inputs['data_attention_mask']}
        o = getattr(model(**inputs), self.args.data_augmentation_attribute)
        if self.args.use_distributional_representation: o = o.exp()
            
    o = self.aug_idxs.proc(o, n_bm=n_bm)

    """
    Preparing augmentation input
    """
    meta_name = f'{self.args.data_aug_meta_name}_meta'
    if (
        meta_name is not None and 
        self.train_dataset.meta is not None and 
        meta_name in self.train_dataset.meta
    ):    
        meta_dset = self.train_dataset.meta[meta_name]
        aug_info  = meta_dset.meta_info
    else:
        raise ValueError(f'Augmentation information not available.')
            
    pad_proc = PadFeatTfm(pad_tok=self.model.config.pad_token_id, prefix="meta")
    info = pad_proc({
        'meta_input_ids':[aug_info['input_ids'][i] for i in o['info2data_idx']], 
        'meta_attention_mask':[aug_info['attention_mask'][i] for i in o['info2data_idx']],
    })

    data_aug_prefix = self.args.data_aug_meta_name if self.args.data_aug_prefix is None else self.args.data_aug_prefix
    
    if self.args.use_augmentation_index_representation:
        rep = torch.tensor(self.aug_idxs.get_items(o['info2data_idx']))
        return {
            f'{data_aug_prefix}2data_idx':o['info2data_idx'],
            f'{data_aug_prefix}2data_data2ptr': o['info2data_data2ptr'],
            f'{data_aug_prefix}2data_meta_repr': rep,
            f'{data_aug_prefix}2data_attention_mask': info['meta_attention_mask'],
        }
    else:
        return {
            f'{data_aug_prefix}2data_idx':o['info2data_idx'], 
            f'{data_aug_prefix}2data_data2ptr': o['info2data_data2ptr'],
            f'{data_aug_prefix}2data_input_ids': info['meta_input_ids'], 
            f'{data_aug_prefix}2data_attention_mask': info['meta_attention_mask'],
        }
    

In [20]:
#| export
@patch
def _perform_generation(self:XCLearner, model:nn.Module, predict_with_generation:Optional[bool]=None):
    model = unwrap_model(model)
    predict_with_generation = self.args.predict_with_generation if predict_with_generation is None else predict_with_generation
    return getattr(model,'use_generation') if hasattr(model,'use_generation') else predict_with_generation

@patch
def _perform_representation(self:XCLearner, model:nn.Module, predict_with_representation:Optional[bool]=None):
    model = unwrap_model(model)
    predict_with_representation = self.args.predict_with_representation if predict_with_representation is None else predict_with_representation
    return getattr(model,'use_representation') if hasattr(model,'use_representation') else predict_with_representation

@patch
def _perform_augmentation(self:XCLearner, model:nn.Module, predict_with_augmentation:Optional[bool]=None):
    model = unwrap_model(model)
    predict_with_augmentation = self.args.predict_with_augmentation if predict_with_augmentation is None else predict_with_augmentation
    return getattr(model,'use_augmentation') if hasattr(model,'use_augmentation') else predict_with_augmentation


In [21]:
#| export
@patch
def resize_pred(cls:XCLearner, t, n_t):
    max_n_t = n_t.max()
    xn_t = max_n_t.max()-n_t+1
    t_ptr = n_t.cumsum(dim=0)-1
    r_t = torch.ones((len(t),), dtype=xn_t.dtype, device=xn_t.device).scatter(0, t_ptr, xn_t)
    xt = t.repeat_interleave(r_t).view(len(n_t), -1)
    return xt

@patch
def output_mask(cls:XCLearner, n_t, l):
    max_n_t = n_t.max()
    xn_t = max_n_t.max()-n_t+1
    t_ptr = n_t.cumsum(dim=0)-1
    mask_ptr = t_ptr+torch.arange(len(t_ptr), device=t_ptr.device)+1
    mask = torch.ones((l+len(n_t),), dtype=mask_ptr.dtype, device=mask_ptr.device).scatter(0, mask_ptr, 0)
    r_mask = torch.ones((l+len(n_t),), dtype=mask_ptr.dtype, device=mask_ptr.device).scatter(0, mask_ptr, xn_t-1)
    mask = mask.repeat_interleave(r_mask).view(len(n_t), -1)
    return mask

@patch
def resize_output(cls:XCLearner, pred_idx, pred_score, pred_ptr):
    return cls.resize_pred(pred_idx, pred_ptr), cls.resize_pred(pred_score, pred_ptr), cls.output_mask(pred_ptr, len(pred_idx)), pred_ptr

@patch
def concatenate_output(cls:XCLearner, gen_o:Dict, repr_o:Dict):
    gen_o['pred_score'] = torch.exp(gen_o['pred_score'])*cls.args.output_concatenation_weight
    gen_o, repr_o = cls.resize_output(**gen_o), cls.resize_output(**repr_o)
    pred_idx, pred_score, mask = [torch.hstack([gen_o[i], repr_o[i].cpu()]).flatten() for i in range(3)]
    idx = torch.where(mask)[0]
    return {
        'pred_idx': pred_idx[idx],
        'pred_score': pred_score[idx],
        'pred_ptr': gen_o[3]+repr_o[3].cpu(),
    }
    

In [22]:
#| export
@patch
def prediction_step(
    self:XCLearner,
    model: nn.Module,
    inputs: Dict[str, Union[torch.Tensor, Any]],
    prediction_loss_only: bool,
    predict_with_generation: bool,
    predict_with_representation: bool,
    predict_with_augmentation:Optional[bool]=None,
    ignore_keys: Optional[List[str]] = None,
    **kwargs,
) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
    with torch.no_grad():
        with self.compute_loss_context_manager(): outputs = model(**inputs)
        if isinstance(outputs, dict):
            loss = outputs["loss"] if "loss" in outputs else None
        else:
            loss = outputs[0]
        if loss is not None: loss = loss.mean().detach()            
    prediction_loss_only = self.args.prediction_loss_only if prediction_loss_only is None else prediction_loss_only
    if prediction_loss_only: return loss, {}
    
    if self._perform_augmentation(model, predict_with_augmentation): 
        aug_inputs = self.augmentation_output(model, inputs, **kwargs)
        inputs.update(aug_inputs)
        
    output, gen_o, repr_o = None, None, None
    if self._perform_generation(model, predict_with_generation): gen_o = self.generation_output(model, inputs, **kwargs)
    if self._perform_representation(model, predict_with_representation): repr_o = self.representation_output(model, inputs, **kwargs)
    
    if gen_o is not None and repr_o is not None:
        output = {f'{k}_gen':v for k,v in gen_o.items()}
        output.update({f'{k}_repr':v for k,v in repr_o.items()})
        output.update(self.concatenate_output(gen_o, repr_o))
    else:
        output = gen_o if repr_o is None else repr_o
        
    labels = {'targ_idx':inputs[self.args.target_indices_key], 'targ_ptr':inputs[self.args.target_pointer_key]} if self.args.target_indices_key in inputs else None
    if labels is not None: output.update(labels)
    
    return loss, output
    

In [23]:
#| export
@patch
def evaluation_loop(
    self:XCLearner,
    dataloader:DataLoader,
    description:str,
    prediction_loss_only:Optional[bool] = None,
    predict_with_generation:Optional[bool]=None,
    predict_with_representation:Optional[bool]=None,
    ignore_keys:Optional[List[str]] = None,
    metric_key_prefix:str="eval",
) -> XCEvalLoopOutput:
    args = self.args
    prediction_loss_only = prediction_loss_only if prediction_loss_only is not None else args.prediction_loss_only

    """
    Disable random addition of noise
    """
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        use_noise = self.model.disable_noise()

    model = self._wrap_model(self.model, training=False, dataloader=dataloader)

    if len(self.accelerator._models) == 0 and model is self.model:
        model = self.accelerator.prepare(model) if self.is_deepspeed_enabled else self.accelerator.prepare_model(model, evaluation_mode=True)
        if self.is_fsdp_enabled: self.model = model
        if model is not self.model: self.model_wrapped = model
        if self.is_deepspeed_enabled: self.deepspeed = self.model_wrapped

    batch_size = self.args.eval_batch_size

    model.eval()
    
    self.callback_handler.eval_dataloader = dataloader
    eval_dataset = getattr(dataloader, "dataset", None)
    
    if self._perform_representation(unwrap_model(model)) and not prediction_loss_only: 
        self._build_lbl_index(eval_dataset)
            
    if self._perform_augmentation(unwrap_model(model)) and not prediction_loss_only: 
        self._build_aug_index(eval_dataset)
    
    if args.past_index >= 0: self._past = None

    losses_host, all_losses = None, None
    host_output, all_output = {}, {}
    
    observed_num_examples = 0
    for step, inputs in enumerate(dataloader):
        observed_batch_size = find_batch_size(inputs)
        if observed_batch_size is not None:
            observed_num_examples += observed_batch_size
            if batch_size is None: batch_size = observed_batch_size
                
        loss, output = self.prediction_step(model, inputs, prediction_loss_only, predict_with_generation, predict_with_representation, ignore_keys=ignore_keys)
        
        if loss is not None:
            losses = self.gather_function((loss.repeat(batch_size)))
            losses_host = losses if losses_host is None else nested_concat(losses_host, losses, padding_index=-100)
        for k in output: host_output[k] = self._gather_host_output(output[k], host_output.get(k, None))
            
        self.control = self.callback_handler.on_prediction_step(args, self.state, self.control)
        
        if args.eval_accumulation_steps is not None and (step + 1) % args.eval_accumulation_steps == 0:
            if losses_host is not None: all_losses = losses_host if all_losses is None else nested_concat(all_losses, losses, padding_index=-100)
            for k in host_output: all_output[k], host_output[k] = self._gather_all_output(host_output[k], all_output.get(k, None)), None
    
    self.gather_function = self.accelerator.gather_for_metrics
    if args.past_index and hasattr(self, "_past"): delattr(self, "_past")

    if losses_host is not None: all_losses = losses_host if all_losses is None else nested_concat(all_losses, losses, padding_index=-100)
    for k in host_output: all_output[k], host_output[k] = self._gather_all_output(host_output[k], all_output.get(k, None)), None
        
    if has_length(eval_dataset): num_samples = len(eval_dataset)
    elif isinstance(eval_dataset, IterableDatasetShard) and getattr(eval_dataset, "num_examples", 0) > 0:
        num_samples = eval_dataset.num_examples
    else:
        if has_length(dataloader): num_samples = self.num_examples(dataloader)
        else: num_samples = observed_num_examples
    if num_samples == 0 and observed_num_examples > 0: num_samples = observed_num_examples
        
    gen_output, repr_output = None, None
    metric_input_keys = ['targ_idx', 'targ_ptr', 'pred_idx', 'pred_ptr', 'pred_score']
    if 'pred_idx_gen' in all_output and all_output['pred_idx_gen'] is not None:
        gen_output = {o:all_output[f'{o}_gen' if o.startswith('pred_') else o] for o in metric_input_keys}
    if 'pred_idx_repr' in all_output and all_output['pred_idx_repr'] is not None:
        repr_output = {o:all_output[f'{o}_repr' if o.startswith('pred_') else o] for o in metric_input_keys}
    

    if (self.compute_metrics is not None and 
        'targ_idx' in all_output and all_output['targ_idx'] is not None and 
        'pred_idx' in all_output and all_output['pred_idx'] is not None):
        
        metrics = self.compute_metrics(**{o:all_output[o] for o in metric_input_keys})
        if gen_output is not None:
            m = self.compute_metrics(**gen_output)
            metrics.update({f'{k}_GEN':v for k,v in m.items()})
        if repr_output is not None:
            m = self.compute_metrics(**repr_output)
            metrics.update({f'{k}_REPR':v for k,v in m.items()})      
    else: metrics = {}
        
    metrics = denumpify_detensorize(metrics)

    if all_losses is not None: metrics[f"{metric_key_prefix}_loss"] = all_losses.mean().item()
    if hasattr(self, "jit_compilation_time"): metrics[f"{metric_key_prefix}_jit_compilation_time"] = self.jit_compilation_time
        
    for key in list(metrics.keys()):
        if not key.startswith(f"{metric_key_prefix}_"): metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

    """
    Set the noise addition state back
    """
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        self.model.set_noise(use_noise)
    
    return XCEvalLoopOutput(pred_idx=all_output.get('pred_idx'), pred_ptr=all_output.get('pred_ptr'), 
                            pred_score=all_output.get('pred_score'),targ_idx=all_output.get('targ_idx'), 
                            targ_ptr=all_output.get('targ_ptr'), gen_output=gen_output, repr_output=repr_output,
                            metrics=metrics, num_samples=num_samples)
    

In [24]:
#| export
@patch
def get_meta_representation(self:XCLearner, dataloader: DataLoader, to_cpu:Optional[bool]=True):
    data_host, all_data = None, None
    
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        use_noise = self.model.disable_noise()
    
    for step, inputs in tqdm(enumerate(dataloader), total=len(dataloader)):
        inputs = inputs.to(self.model.device)
        with torch.no_grad(): data = getattr(self.model.get_meta_representation(**inputs), self.args.metadata_representation_attribute)
        data_host = self._gather_host_output(data, data_host)
        if self.args.representation_accumulation_steps is not None and (step + 1) % self.args.representation_accumulation_steps == 0:
            all_data, data_host = self._gather_all_output(data_host, all_data, to_cpu=to_cpu), None
            
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        self.model.set_noise(use_noise)
            
    return self._gather_all_output(data_host, all_data, to_cpu=to_cpu)

@patch
def get_representation(self:XCLearner, dataloader: DataLoader, representation_attribute:str, to_cpu:Optional[bool]=True):
    data_host, all_data = None, None
    
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        use_noise = self.model.disable_noise()
    
    for step, inputs in tqdm(enumerate(dataloader), total=len(dataloader)):
        inputs = inputs.to(self.model.device)
        with torch.no_grad(): data = getattr(self.model(**inputs), representation_attribute)
        data_host = self._gather_host_output(data, data_host)
        if self.args.representation_accumulation_steps is not None and (step + 1) % self.args.representation_accumulation_steps == 0:
            all_data, data_host = self._gather_all_output(data_host, all_data, to_cpu=to_cpu), None
            
    if hasattr(self.model, 'disable_noise') and callable(getattr(self.model, 'disable_noise')):
        self.model.set_noise(use_noise)
            
    return self._gather_all_output(data_host, all_data, to_cpu=to_cpu)
    

In [25]:
from xcai.optimizers.oakX import MultipleOptimizer, MultipleScheduler

@patch
def create_optimizer_and_scheduler(self:XCLearner, num_training_steps: int):
    NO_DECAY = ['bias', 'LayerNorm.weight']

    dense, sparse = [], []
    for k, p in model.named_parameters():
        if p.requires_grad:
            if "meta_embeddings" not in k: dense.append((k,p))
            else: sparse.append(p)

    params = [
        {'params': [p for n, p in dense if not any(nd in n for nd in NO_DECAY)], 'weight_decay': 0.01},
        {'params': [p for n, p in dense if any(nd in n for nd in NO_DECAY)], 'weight_decay': 0.0},
    ]

    optimizer_list = [torch.optim.AdamW(params, **{'lr': self.args.learning_rate, 'eps': 1e-6}),
                      torch.optim.SparseAdam(sparse, **{'lr': self.args.learning_rate * self.args.free_parameter_lr_coefficient, 'eps': 1e-6})]

    self.optimizer = MultipleOptimizer(optimizer_list)
    scheduler_list = [transformers.get_linear_schedule_with_warmup(self.optimizer.optimizers[0], num_warmup_steps=self.args.warmup_steps, num_training_steps=num_training_steps),
                      transformers.get_cosine_schedule_with_warmup(self.optimizer.optimizers[1], num_warmup_steps=self.args.free_parameter_warmup_steps, num_training_steps=num_training_steps)]
    
    self.lr_scheduler = MultipleScheduler(scheduler_list)

    super().create_optimizer_and_scheduler(num_training_steps)
    

### Training loop

In [26]:
#| export
@patch
def _get_train_sampler(self:XCLearner):
    if self.train_dataset is None or not has_length(self.train_dataset):
        return None
        
    if self.args.group_by_length:
        if is_datasets_available() and isinstance(self.train_dataset, datasets.Dataset):
            lengths = (
                self.train_dataset[self.args.length_column_name]
                if self.args.length_column_name in self.train_dataset.column_names
                else None
            )
        else:
            lengths = None
        model_input_name = self.tokenizer.model_input_names[0] if self.tokenizer is not None else None
        return LengthGroupedSampler(
            self.args.train_batch_size * self.args.gradient_accumulation_steps,
            dataset=self.train_dataset,
            lengths=lengths,
            model_input_name=model_input_name,
        )

    elif self.args.group_by_cluster:
        return ClusterGroupedSampler(n=len(self.train_dataset))
    else:
        return RandomSampler(self.train_dataset)
        

In [27]:
#| export
@patch
def get_train_dataloader(self:XCLearner):
    if self.train_dataset is None:
        raise ValueError("Trainer: training requires a train_dataset.")

    train_dataset = self.train_dataset
    data_collator = self.data_collator
    if is_datasets_available() and isinstance(train_dataset, datasets.Dataset):
        train_dataset = self._remove_unused_columns(train_dataset, description="training")
    else:
        data_collator = self._get_collator_with_removed_columns(data_collator, description="training")

    dataloader_params = {
        "batch_size": self._train_batch_size,
        "collate_fn": data_collator,
        "num_workers": self.args.dataloader_num_workers,
        "pin_memory": self.args.dataloader_pin_memory,
        "persistent_workers": self.args.dataloader_persistent_workers,
    }

    if not isinstance(train_dataset, torch.utils.data.IterableDataset):
        dataloader_params["sampler"] = self._get_train_sampler()
        dataloader_params["drop_last"] = self.args.dataloader_drop_last
        dataloader_params["worker_init_fn"] = seed_worker
        dataloader_params["prefetch_factor"] = self.args.dataloader_prefetch_factor
    
    return DataLoader(train_dataset, **dataloader_params)
    

In [28]:
#| export
@patch
def _get_min_cluster_sz(self:XCLearner, epochs_trained:int, num_train_epochs:int):
    
    if self.args.num_clustering_warmup_epochs is not None:
        if epochs_trained < self.args.num_clustering_warmup_epochs: return None
        else: epochs_trained -= self.args.num_clustering_warmup_epochs
    
    if self.args.clustering_type == 'LINEAR':
        if self.args.maximum_clusters is None: return self.train_dataset.n_data//self.args.minimum_clusters
        else:
            n_cluster = (self.args.maximum_clusters-self.args.minimum_clusters)/num_train_epochs*epochs_trained
            return self.train_dataset.n_data//int(self.args.minimum_clusters+n_cluster)
        
    elif self.args.clustering_type == 'EXPO':
        mult = 2**(epochs_trained//self.args.num_cluster_size_update_epochs)
        cluster_sz = self.args.minimum_cluster_size*mult
        cluster_sz = (
            self.args.maximum_cluster_size 
            if self.args.maximum_cluster_size is not None and cluster_sz > self.args.maximum_cluster_size 
            else cluster_sz
        )
        return cluster_sz
    
    else: raise ValueError(f'Invalid `clustering_type`({self.args.clustering_type}).')
    

In [ ]:
#| export
@patch
def _get_train_data_cluster(self:XCLearner, epochs_trained:int, num_train_epochs:int):
    with torch.no_grad():
        dataset = self._get_dataset(self.train_dataset, dset_type='data', use_metadata=self.args.use_data_metadata_for_clustering)
        dataloader = self.get_test_dataloader(dataset)
        data_repr = self.get_representation(dataloader, representation_attribute=self.args.clustering_representation_attribute, 
                                            to_cpu=self.args.use_cpu_for_clustering)
        if self.args.use_distributional_representation: data_repr = data_repr.exp()
            
        cluster = BalancedClusters.proc(data_repr, self._get_min_cluster_sz(epochs_trained, num_train_epochs), 
                                        clustering_devices=self.args.clustering_devices)
    return cluster

@patch
def update_dataloader_sampler(self:XCLearner, dataloader:DataLoader, epochs_trained:int, num_train_epochs:int):
    if isinstance(dataloader.sampler, ClusterGroupedSampler):
        self.model.train()
        cluster = self._get_train_data_cluster(epochs_trained, num_train_epochs)
        dataloader.sampler.set_cluster(cluster)
    

In [ ]:
#| export
@patch
def prune_metadata(self:XCLearner):
    if self.train_dataset.meta is None: return

    self.model.train()
    with torch.no_grad():
        data_dset = self._get_dataset(self.train_dataset, dset_type='data', use_metadata=self.args.use_data_metadata_for_pruning)
        dataloader = self.get_test_dataloader(data_dset)
        data_repr = self.get_representation(dataloader, representation_attribute=self.args.output_representation_attribute)
    
        lbl_dset = self._get_dataset(self.train_dataset, dset_type='lbl', use_metadata=self.args.use_label_metadata)
        dataloader = self.get_test_dataloader(lbl_dset)
        lbl_repr = self.get_representation(dataloader, representation_attribute=self.args.label_representation_attribute)
    
        prune_metadata_names = list(self.train_dataset.meta.keys()) if self.args.prune_metadata_names is None else self.args.prune_metadata_names
        for m in self.args.prune_metadata_names:
            if m not in self.train_dataset.meta: raise ValueError(f'Invalid metadata name: {m}')
                
            meta_dset = self.train_dataset.meta[m]
            dataloader = self.get_test_dataloader(type(self.train_dataset.data)(data_info=meta_dset.meta_info))
            meta_repr = self.get_representation(dataloader, representation_attribute=self.args.metadata_representation_attribute)
            
            meta_dset.prune_data_meta(data_repr, meta_repr, batch_size=self.args.metadata_prune_batch_size, 
                                      thresh=self.args.prune_metadata_threshold, topk=self.args.prune_metadata_topk)
            meta_dset.prune_lbl_meta(lbl_repr, meta_repr, batch_size=self.args.metadata_prune_batch_size, 
                                     thresh=self.args.prune_metadata_threshold, topk=self.args.prune_metadata_topk)
            

In [ ]:
#| export
@patch
def get_aug_data_meta(self:XCLearner, data_repr:torch.Tensor, batch_size:Optional[int]=64):
    data_repr = F.normalize(data_repr, dim=1)
    dl = DataLoader(data_repr, batch_size=batch_size, shuffle=False)
    
    indices,num_items = None,None
    for b in tqdm(dl, total=len(dl)): 
        o = self.aug_idxs.proc(b, n_bm=self.args.augmentation_num_beams)
        indices = o['info2data_idx'] if indices is None else torch.hstack([indices,o['info2data_idx']])
        num_items = o['info2data_data2ptr'] if num_items is None else torch.hstack([num_items,o['info2data_data2ptr']])
    indptr = torch.concat([torch.tensor([0]), num_items.cumsum(dim=0)])
    data = torch.ones((indices.shape[0],))
    
    data_meta = sparse.csr_matrix((data, indices, indptr), shape=(data_repr.shape[0], self.aug_idxs.index.element_count))
    return data_meta
    

In [ ]:
#| export
@patch
def get_augmentation_metadata(self:XCLearner):
    self.model.train()
    meta_name = f'{self.args.data_aug_meta_name}_meta' if self.args.data_aug_meta_name is not None else None
    if (
        self.train_dataset.meta is None or 
        meta_name is None or 
        meta_name not in self.train_dataset.meta
    ): 
        return

    with torch.no_grad():
        dataloader = self.get_test_dataloader(self.train_dataset.data_dset)
        data_repr = self.get_representation(dataloader, representation_attribute=self.args.data_augmentation_attribute)
        
        self.aug_idxs = IndexSearch(space=self.args.index_space, efc=self.args.index_efc, m=self.args.index_m, 
                                    efs=self.args.index_efs, n_bm=self.args.augmentation_num_beams, 
                                    n_threads=self.args.index_num_threads)
        
        meta_info = getattr(self.train_dataset.meta[meta_name], 'meta_info')
        dataloader = self.get_test_dataloader(type(self.train_dataset.data)(data_info=meta_info))
        meta_repr = self.get_meta_representation(dataloader, to_cpu=isinstance(self.aug_idxs, IndexSearch))
        self.aug_idxs.build(meta_repr)
    
        data_meta = self.get_aug_data_meta(data_repr, batch_size=self.args.data_meta_batch_size)
        data_aug_prefix = self.args.data_aug_meta_name if self.args.data_aug_prefix is None else self.args.data_aug_prefix

    meta_info = {
        'identifier': meta_info['identifier'],
        'input_text': meta_info['input_text'],
        'meta_repr': meta_repr.tolist(),
        'attention_mask': meta_info['attention_mask'],
    }
    
    metadata = type(self.train_dataset.meta)(prefix=data_aug_prefix, data_meta=data_meta, lbl_meta=sparse.csr_matrix((self.train_dataset.n_lbl, meta_repr.shape[0])),
                                             meta_info=meta_info)
    meta_dset = self.train_dataset.meta[meta_name]
    
    if isinstance(metadata, SMetaXCDataset):
        metadata.n_data_meta_samples = meta_dset.n_data_meta_samples
        metadata.n_sdata_meta_samples = meta_dset.n_sdata_meta_samples
        metadata.meta_oversample = meta_dset.meta_oversample
    elif isinstance(metadata, MetaXCDataset):
        metadata.n_data_meta_samples = self.args.augmentation_num_beams
    else: raise ValueError('Invalid meta dataset.')
            
    self.train_dataset.meta[f'{data_aug_prefix}_meta'] = metadata
    

In [ ]:
#| export
@patch
def _get_data_representation(self:XCLearner, dataset:Optional[Dataset]=None, to_cpu:Optional[bool]=True):
    if dataset is not None:
        dataset = self._get_dataset(dataset, dset_type='data', use_metadata=self.args.use_data_metadata_for_representation)
        dataloader = self.get_test_dataloader(dataset)
        data_rep = self.get_representation(dataloader, representation_attribute=self.args.output_representation_attribute, to_cpu=to_cpu)
        return data_rep
    else:
        raise ValueError('`dataset` is None, could not create data representation.')
        

In [ ]:
#| export
@patch
def get_data_and_lbl_representation(self:XCLearner, dataset:Optional[Dataset], to_cpu:Optional[bool]=True):
    with torch.no_grad():
        lbl_rep,data_rep = self._get_lbl_representation(dataset, to_cpu=to_cpu), self._get_data_representation(dataset, to_cpu=to_cpu)
    return data_rep,lbl_rep
    

In [ ]:
#| export
@patch
def mix_metadata(self:XCLearner, epochs_trained:int):
    if epochs_trained == self.args.maximum_mix_metadata_epochs: pct = 1
    else: pct = (epochs_trained - self.args.num_mix_metadata_warmup_epochs)/(self.args.maximum_mix_metadata_epochs - self.args.num_mix_metadata_warmup_epochs)
        
    if pct < 0: raise ValueError(f'Mixing percentage cannot be negative: {pct:.2f}')
    self.train_dataset.mix_meta_dataset(self.args.mix_metadata_name_1, self.args.mix_metadata_name_2, 
                                        pct=pct, k=self.args.mix_metadata_k)
    

In [ ]:
#| export
@patch
def _validate_group_by_cluster(self:XCLearner):
    if self.args.group_by_cluster and (not hasattr(self.model,'use_representation') or  not getattr(unwrap_model(self.model),'use_representation')):
        raise ValueError('Cannot use `group_by_cluster` for models without `use_representation`.')
        self.args.group_by_cluster = False

@patch
def _inner_training_loop(
    self:XCLearner, batch_size=None, args=None, resume_from_checkpoint=None, trial=None, ignore_keys_for_eval=None
):
    self.accelerator.free_memory()
    self._train_batch_size = batch_size
    if self.args.auto_find_batch_size:
        if self.state.train_batch_size != self._train_batch_size:
            from accelerate.utils import release_memory

            (self.model_wrapped,) = release_memory(self.model_wrapped)
            self.model_wrapped = self.model

            # Check for DeepSpeed *after* the intial pass and modify the config
            if self.is_deepspeed_enabled:
                # Temporarily unset `self.args.train_batch_size`
                original_bs = self.args.per_device_train_batch_size
                self.args.per_device_train_batch_size = self._train_batch_size // max(1, self.args.n_gpu)
                self.propagate_args_to_deepspeed(True)
                self.args.per_device_train_batch_size = original_bs
        self.state.train_batch_size = self._train_batch_size
    logger.debug(f"Currently training with a batch size of: {self._train_batch_size}")
    
    # Data loader and number of training steps
    self._validate_group_by_cluster()
    train_dataloader = self.get_train_dataloader()
    
    if self.is_fsdp_xla_v2_enabled:
        train_dataloader = tpu_spmd_dataloader(train_dataloader)

    # Setting up training control variables:
    # number of training epochs: num_train_epochs
    # number of training steps per epoch: num_update_steps_per_epoch
    # total number of training steps to execute: max_steps
    total_train_batch_size = self._train_batch_size * args.gradient_accumulation_steps * args.world_size

    len_dataloader = None
    num_train_tokens = None
    if has_length(train_dataloader):
        len_dataloader = len(train_dataloader)
        num_update_steps_per_epoch = len_dataloader // args.gradient_accumulation_steps
        num_update_steps_per_epoch = max(num_update_steps_per_epoch, 1)
        num_examples = self.num_examples(train_dataloader)
        if args.max_steps > 0:
            max_steps = args.max_steps
            num_train_epochs = args.max_steps // num_update_steps_per_epoch + int(
                args.max_steps % num_update_steps_per_epoch > 0
            )
            # May be slightly incorrect if the last batch in the training dataloader has a smaller size but it's
            # the best we can do.
            num_train_samples = args.max_steps * total_train_batch_size
            if args.include_tokens_per_second:
                num_train_tokens = (
                    self.num_tokens(train_dataloader, args.max_steps) * args.gradient_accumulation_steps
                )
        else:
            max_steps = math.ceil(args.num_train_epochs * num_update_steps_per_epoch)
            num_train_epochs = math.ceil(args.num_train_epochs)
            num_train_samples = self.num_examples(train_dataloader) * args.num_train_epochs
            if args.include_tokens_per_second:
                num_train_tokens = self.num_tokens(train_dataloader) * args.num_train_epochs
    elif args.max_steps > 0:  # Rely on max_steps when dataloader does not have a working size
        max_steps = args.max_steps
        # Setting a very large number of epochs so we go as many times as necessary over the iterator.
        num_train_epochs = sys.maxsize
        num_update_steps_per_epoch = max_steps
        num_examples = total_train_batch_size * args.max_steps
        num_train_samples = args.max_steps * total_train_batch_size
        if args.include_tokens_per_second:
            num_train_tokens = self.num_tokens(train_dataloader, args.max_steps) * args.gradient_accumulation_steps
    else:
        raise ValueError(
            "args.max_steps must be set to a positive value if dataloader does not have a length, was"
            f" {args.max_steps}"
        )

    if DebugOption.UNDERFLOW_OVERFLOW in self.args.debug:
        if self.args.n_gpu > 1:
            # nn.DataParallel(model) replicates the model, creating new variables and module
            # references registered here no longer work on other gpus, breaking the module
            raise ValueError(
                "Currently --debug underflow_overflow is not supported under DP. Please use DDP"
                " (torchrun or torch.distributed.launch (deprecated))."
            )
        else:
            debug_overflow = DebugUnderflowOverflow(self.model)  # noqa

    delay_optimizer_creation = is_sagemaker_mp_enabled() or self.is_fsdp_xla_enabled or self.is_fsdp_enabled

    # We need to reset the scheduler, as its parameters may be different on subsequent calls
    if self._created_lr_scheduler:
        self.lr_scheduler = None
        self._created_lr_scheduler = False

    if self.is_deepspeed_enabled:
        self.optimizer, self.lr_scheduler = deepspeed_init(self, num_training_steps=max_steps)

    if not delay_optimizer_creation:
        self.create_optimizer_and_scheduler(num_training_steps=max_steps)

    self.state = TrainerState(
            stateful_callbacks=[
                cb for cb in self.callback_handler.callbacks + [self.control] if isinstance(cb, ExportableState)
            ]
        )
    self.state.is_hyper_param_search = trial is not None
    self.state.train_batch_size = self._train_batch_size

    # Compute absolute values for logging, eval, and save if given as ratio
    if args.logging_steps is not None:
        if args.logging_steps < 1:
            self.state.logging_steps = math.ceil(max_steps * args.logging_steps)
        else:
            self.state.logging_steps = args.logging_steps
    if args.eval_steps is not None:
        if args.eval_steps < 1:
            self.state.eval_steps = math.ceil(max_steps * args.eval_steps)
        else:
            self.state.eval_steps = args.eval_steps
    if args.save_steps is not None:
        if args.save_steps < 1:
            self.state.save_steps = math.ceil(max_steps * args.save_steps)
        else:
            self.state.save_steps = args.save_steps

    # Activate gradient checkpointing if needed
    if args.gradient_checkpointing:
        if args.gradient_checkpointing_kwargs is None:
            gradient_checkpointing_kwargs = {}
        else:
            gradient_checkpointing_kwargs = args.gradient_checkpointing_kwargs

        self.model.gradient_checkpointing_enable(gradient_checkpointing_kwargs=gradient_checkpointing_kwargs)

    model = self._wrap_model(self.model_wrapped)

    # as the model is wrapped, don't use `accelerator.prepare`
    # this is for unhandled cases such as
    # FSDP-XLA, SageMaker MP/DP, DataParallel, IPEX
    use_accelerator_prepare = True if model is self.model else False

    if delay_optimizer_creation:
        if use_accelerator_prepare:
            self.model = self.accelerator.prepare(self.model)
        self.create_optimizer_and_scheduler(num_training_steps=max_steps)

    # prepare using `accelerator` prepare
    if use_accelerator_prepare:
        self.model.train()
        if hasattr(self.lr_scheduler, "step"):
            if self.use_apex:
                model = self.accelerator.prepare(self.model)
            else:
                model, self.optimizer = self.accelerator.prepare(self.model, self.optimizer)
        else:
            # to handle cases wherein we pass "DummyScheduler" such as when it is specified in DeepSpeed config.
            model, self.optimizer, self.lr_scheduler = self.accelerator.prepare(
                self.model, self.optimizer, self.lr_scheduler
            )

    if self.is_fsdp_enabled:
        self.model = self.model_wrapped = model

    # for the rest of this function `model` is the outside model, whether it was wrapped or not
    if model is not self.model:
        self.model_wrapped = model

    # backward compatibility
    if self.is_deepspeed_enabled:
        self.deepspeed = self.model_wrapped

    # ckpt loading
    if resume_from_checkpoint is not None:
        if self.is_deepspeed_enabled:
            deepspeed_load_checkpoint(
                self.model_wrapped, resume_from_checkpoint, load_module_strict=not _is_peft_model(self.model)
            )
        elif is_sagemaker_mp_enabled() or self.is_fsdp_enabled:
            self._load_from_checkpoint(resume_from_checkpoint, self.model_wrapped)

    # Check if saved optimizer or scheduler states exist
    self._load_optimizer_and_scheduler(resume_from_checkpoint)

    # important: at this point:
    # self.model         is the Transformers Model
    # self.model_wrapped is DDP(Transformers Model), Deepspeed(Transformers Model),
    # FSDP(Transformers Model), Dynamo Optimized Module(Transformers Model) etc.

    # Train!
    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {num_examples:,}")
    logger.info(f"  Num Epochs = {num_train_epochs:,}")
    logger.info(f"  Instantaneous batch size per device = {self.args.per_device_train_batch_size:,}")
    if self.args.per_device_train_batch_size != self._train_batch_size:
        logger.info(f"  Training with DataParallel so batch size has been adjusted to: {self._train_batch_size:,}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_train_batch_size:,}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {max_steps:,}")
    logger.info(f"  Number of trainable parameters = {get_model_param_count(model, trainable_only=True):,}")

    self.state.epoch = 0
    start_time = time.time()
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    steps_trained_progress_bar = None

    # Check if continuing training from a checkpoint
    if resume_from_checkpoint is not None and os.path.isfile(
        os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME)
    ):
        self.state = TrainerState.load_from_json(os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME))
        epochs_trained = self.state.global_step // num_update_steps_per_epoch
        if not args.ignore_data_skip:
            steps_trained_in_current_epoch = self.state.global_step % (num_update_steps_per_epoch)
            steps_trained_in_current_epoch *= args.gradient_accumulation_steps
        else:
            steps_trained_in_current_epoch = 0

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info(f"  Continuing training from epoch {epochs_trained}")
        logger.info(f"  Continuing training from global step {self.state.global_step}")
        if not args.ignore_data_skip:
            logger.info(
                f"  Will skip the first {epochs_trained} epochs then the first"
                f" {steps_trained_in_current_epoch} batches in the first epoch."
            )

    # Update the references
    self.callback_handler.model = self.model
    self.callback_handler.optimizer = self.optimizer
    self.callback_handler.lr_scheduler = self.lr_scheduler
    self.callback_handler.train_dataloader = train_dataloader
    if self.hp_name is not None and self._trial is not None:
        # use self._trial because the SigOpt/Optuna hpo only call `_hp_search_setup(trial)` instead of passing trial
        # parameter to Train when using DDP.
        self.state.trial_name = self.hp_name(self._trial)
    if trial is not None:
        assignments = trial.assignments if self.hp_search_backend == HPSearchBackend.SIGOPT else trial
        self.state.trial_params = hp_params(assignments)
    else:
        self.state.trial_params = None
    # This should be the same if the state has been saved but in case the training arguments changed, it's safer
    # to set this after the load.
    self.state.max_steps = max_steps
    self.state.num_train_epochs = num_train_epochs
    self.state.is_local_process_zero = self.is_local_process_zero()
    self.state.is_world_process_zero = self.is_world_process_zero()

    # tr_loss is a tensor to avoid synchronization of TPUs through .item()
    tr_loss = torch.tensor(0.0).to(args.device)
    # _total_loss_scalar is updated everytime .item() has to be called on tr_loss and stores the sum of all losses
    self._total_loss_scalar = 0.0
    self._globalstep_last_logged = self.state.global_step
    model.zero_grad()
    grad_norm: Optional[float] = None

    self.control = self.callback_handler.on_train_begin(args, self.state, self.control)

    # Skip the first epochs_trained epochs to get the random state of the dataloader at the right point.
    if not args.ignore_data_skip:
        for epoch in range(epochs_trained):
            sampler = get_dataloader_sampler(train_dataloader)
            sampler_kinds = [RandomSampler]
            if version.parse(accelerate_version) > version.parse("0.23.0"):
                sampler_kinds.append(SeedableRandomSampler)
            is_random_sampler = isinstance(sampler, tuple(sampler_kinds))
            if not is_random_sampler:
                # We just need to begin an iteration to create the randomization of the sampler.
                for _ in train_dataloader:
                    break
            else:
                # Otherwise we need to call the whooooole sampler cause there is some random operation added
                # AT THE VERY END!
                sampler = sampler if sampler is not None else []
                _ = list(sampler)

    total_batched_samples = 0
    for epoch in range(epochs_trained, num_train_epochs):

        if self.args.augment_metadata and (epoch % self.args.num_metadata_augment_epochs == 0 or epoch == self.args.num_metadata_augment_warmup_epochs) and epoch >= self.args.num_metadata_augment_warmup_epochs:
            self.accelerator.free_memory()
            self.get_augmentation_metadata()
            self.accelerator.free_memory()
            
        if self.args.group_by_cluster and (epoch % self.args.num_cluster_update_epochs == 0 or epoch == self.args.num_clustering_warmup_epochs) and epoch >= self.args.num_clustering_warmup_epochs:
            self.accelerator.free_memory()
            self.update_dataloader_sampler(train_dataloader, epoch, num_train_epochs)
            self.accelerator.free_memory()

        if self.args.prune_metadata and (epoch % self.args.num_metadata_prune_epochs == 0 or epoch == self.args.num_metadata_prune_warmup_epochs) and epoch >= self.args.num_metadata_prune_warmup_epochs: 
            self.accelerator.free_memory()
            self.prune_metadata()
            self.accelerator.free_memory()

        if self.args.mix_metadata and (epoch % self.args.num_mix_metadata_epochs == 0 or epoch == self.args.num_mix_metadata_warmup_epochs or epoch == self.args.maximum_mix_metadata_epochs) and epoch >= self.args.num_mix_metadata_warmup_epochs and epoch <= self.args.maximum_mix_metadata_epochs:
            self.mix_metadata(epoch)

        epoch_iterator = train_dataloader
        if hasattr(epoch_iterator, "set_epoch"):
            epoch_iterator.set_epoch(epoch)

        # Reset the past mems state at the beginning of each epoch if necessary.
        if args.past_index >= 0:
            self._past = None

        steps_in_epoch = (
            len(epoch_iterator)
            if len_dataloader is not None
            else args.max_steps * args.gradient_accumulation_steps
        )
        self.control = self.callback_handler.on_epoch_begin(args, self.state, self.control)

        if epoch == epochs_trained and resume_from_checkpoint is not None and steps_trained_in_current_epoch == 0:
            self._load_rng_state(resume_from_checkpoint)

        rng_to_sync = False
        steps_skipped = 0
        if steps_trained_in_current_epoch > 0:
            epoch_iterator = skip_first_batches(epoch_iterator, steps_trained_in_current_epoch)
            steps_skipped = steps_trained_in_current_epoch
            steps_trained_in_current_epoch = 0
            rng_to_sync = True

        step = -1
        for step, inputs in enumerate(epoch_iterator):
            total_batched_samples += 1

            if self.args.include_num_input_tokens_seen:
                main_input_name = getattr(self.model, "main_input_name", "input_ids")
                if main_input_name not in inputs:
                    logger.warning(
                        "Tried to track the number of tokens seen, however the current model is "
                        "not configured properly to know what item is the input. To fix this, add "
                        "a `main_input_name` attribute to the model class you are using."
                    )
                else:
                    self.state.num_input_tokens_seen += self.accelerator.gather(inputs[main_input_name]).numel()
            if rng_to_sync:
                self._load_rng_state(resume_from_checkpoint)
                rng_to_sync = False

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                if steps_trained_progress_bar is not None:
                    steps_trained_progress_bar.update(1)
                if steps_trained_in_current_epoch == 0:
                    self._load_rng_state(resume_from_checkpoint)
                continue
            elif steps_trained_progress_bar is not None:
                steps_trained_progress_bar.close()
                steps_trained_progress_bar = None

            if step % args.gradient_accumulation_steps == 0:
                self.control = self.callback_handler.on_step_begin(args, self.state, self.control)

            with self.accelerator.accumulate(model):
                tr_loss_step = self.training_step(model, inputs)

            if (
                args.logging_nan_inf_filter
                and not is_torch_xla_available()
                and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
            ):
                # if loss is nan or inf simply add the average of previous logged losses
                tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_last_logged)
            else:
                tr_loss += tr_loss_step

            self.current_flos += float(self.floating_point_ops(inputs))

            is_last_step_and_steps_less_than_grad_acc = (
                steps_in_epoch <= args.gradient_accumulation_steps and (step + 1) == steps_in_epoch
            )

            if (
                total_batched_samples % args.gradient_accumulation_steps == 0
                or
                # last step in epoch but step is always smaller than gradient_accumulation_steps
                is_last_step_and_steps_less_than_grad_acc
            ):
                # the `or` condition of `is_last_step_and_steps_less_than_grad_acc` is not covered
                # in accelerate. So, explicitly enable sync gradients to True in that case.
                if is_last_step_and_steps_less_than_grad_acc:
                    self.accelerator.gradient_state._set_sync_gradients(True)

                # Gradient clipping
                if args.max_grad_norm is not None and args.max_grad_norm > 0:
                    # deepspeed does its own clipping

                    if is_sagemaker_mp_enabled() and args.fp16:
                        _grad_norm = self.optimizer.clip_master_grads(args.max_grad_norm)
                    elif self.use_apex:
                        # Revert to normal clipping otherwise, handling Apex or full precision
                        _grad_norm = nn.utils.clip_grad_norm_(
                            amp.master_params(self.optimizer),
                            args.max_grad_norm,
                        )
                    else:
                        _grad_norm = self.accelerator.clip_grad_norm_(
                            model.parameters(),
                            args.max_grad_norm,
                        )

                    if (
                        is_accelerate_available()
                        and self.accelerator.distributed_type == DistributedType.DEEPSPEED
                    ):
                        grad_norm = model.get_global_grad_norm()
                    else:
                        grad_norm = _grad_norm.item() if _grad_norm is not None else None

                # Optimizer step
                self.optimizer.step()
                optimizer_was_run = not self.accelerator.optimizer_step_was_skipped
                if optimizer_was_run:
                    # Delay optimizer scheduling until metrics are generated
                    if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                        self.lr_scheduler.step()

                model.zero_grad()
                self.state.global_step += 1
                self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epoch
                self.control = self.callback_handler.on_step_end(args, self.state, self.control)

                self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval, start_time)
            else:
                self.control = self.callback_handler.on_substep_end(args, self.state, self.control)

            if self.control.should_epoch_stop or self.control.should_training_stop:
                # PyTorch/XLA relies on the data loader to insert the mark_step for
                # each step. Since we are breaking the loop early, we need to manually
                # insert the mark_step here.
                if is_torch_xla_available():
                    xm.mark_step()
                break

            if self.args.use_memory_aggressively:
                del inputs
                self.accelerator.free_memory()
                
        if step < 0:
            logger.warning(
                "There seems to be not a single sample in your epoch_iterator, stopping training at step"
                f" {self.state.global_step}! This is expected if you're using an IterableDataset and set"
                f" num_steps ({max_steps}) higher than the number of available samples."
            )
            self.control.should_training_stop = True

        self.control = self.callback_handler.on_epoch_end(args, self.state, self.control)
        self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval, start_time)

        if DebugOption.TPU_METRICS_DEBUG in self.args.debug:
            if is_torch_xla_available():
                # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
                xm.master_print(met.metrics_report())
            else:
                logger.warning(
                    "You enabled PyTorch/XLA debug metrics but you don't have a TPU "
                    "configured. Check your training configuration if this is unexpected."
                )
        if self.control.should_training_stop:
            break

    if args.past_index and hasattr(self, "_past"):
        # Clean the state at the end of training
        delattr(self, "_past")

    logger.info("\n\nTraining completed. Do not forget to share your model on huggingface.co/models =)\n\n")
    if args.load_best_model_at_end and self.state.best_model_checkpoint is not None:
        # Wait for everyone to get here so we are sure the model has been saved by process 0.
        if is_torch_xla_available():
            xm.rendezvous("load_best_model_at_end")
        elif args.parallel_mode == ParallelMode.DISTRIBUTED:
            dist.barrier()
        elif is_sagemaker_mp_enabled():
            smp.barrier()

        self._load_best_model()

    # add remaining tr_loss
    self._total_loss_scalar += tr_loss.item()
    train_loss = self._total_loss_scalar / self.state.global_step

    metrics = speed_metrics(
        "train",
        start_time,
        num_samples=num_train_samples,
        num_steps=self.state.max_steps,
        num_tokens=num_train_tokens,
    )
    self.store_flos()
    metrics["total_flos"] = self.state.total_flos
    metrics["train_loss"] = train_loss

    self.is_in_train = False

    self._memory_tracker.stop_and_update_metrics(metrics)

    self.log(metrics)

    run_dir = self._get_output_dir(trial)
    checkpoints_sorted = self._sorted_checkpoints(use_mtime=False, output_dir=run_dir)

    # Delete the last checkpoint when save_total_limit=1 if it's different from the best checkpoint and process allowed to save.
    if self.args.should_save and self.state.best_model_checkpoint is not None and self.args.save_total_limit == 1:
        for checkpoint in checkpoints_sorted:
            if not os.path.samefile(checkpoint, self.state.best_model_checkpoint):
                logger.info(f"Deleting older checkpoint [{checkpoint}] due to args.save_total_limit")
                shutil.rmtree(checkpoint)

    self.control = self.callback_handler.on_train_end(args, self.state, self.control)

    # Wait for the checkpoint to be uploaded.
    self._finish_current_push()

    # After training we make sure to retrieve back the original forward pass method
    # for the embedding layer by removing the forward post hook.
    if self.neftune_noise_alpha is not None:
        self._deactivate_neftune(self.model)

    return TrainOutput(self.state.global_step, train_loss, metrics)
    


### Example

In [ ]:
from xcai.models.PPP0XX import DBT021

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [ ]:
args = XCLearningArguments(
    output_dir='/scratch/scai/phd/aiz218323/scratch/outputs/default/',
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    num_train_epochs=50,
    eval_steps=50,
    weight_decay=0.01,
    evaluation_strategy='steps',
    
    representation_accumulation_steps=100,
    representation_search_type='BRUTEFORCE',
    output_representation_attribute='data_repr',
    label_representation_attribute='data_repr',
    metadata_representation_attribute='data_repr',
    data_augmentation_attribute='data_repr',
    representation_attribute='data_repr',
    
    group_by_cluster=False,
    num_clustering_warmup_epochs=0,
    num_cluster_update_epochs=2,
    num_cluster_size_update_epochs=2,
    use_data_metadata_for_clustering=True,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=4,

    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',

    use_distributional_representation=False,
    use_encoder_parallel=True,
    max_grad_norm=None, 
    fp16=True,

    use_centroid_data_metadata=True,
    use_centroid_label_representation=True,
    centroid_data_attribute_representation='data_fused_repr',
    centroid_data_batch_size=2048,

    use_teacher_lbl_representation=False,
    use_teacher_data_representation=False,

    predict_with_augmentation=True,
    use_augmentation_index_representation=True,
    
    label_names=['cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
                 'cat2lbl2data_idx', 'cat2lbl2data_input_ids', 'cat2lbl2data_attention_mask'],

    prune_metadata=False,
    num_metadata_prune_warmup_epochs=0,
    num_metadata_prune_epochs=1,
    metadata_prune_batch_size=2048,
    prune_metadata_names=['cat_meta'],
    use_data_metadata_for_pruning=False,
    prune_metadata_threshold=0.0,
    prune_metadata_topk=3,
    
    data_aug_meta_name='cat',
    augmentation_num_beams=3,
    data_aug_prefix='cat',
    use_label_metadata=False,
    
    data_meta_batch_size=2048,
    augment_metadata=False,
    num_metadata_augment_warmup_epochs=0,
    num_metadata_augment_epochs=5,

    mix_metadata=True,
    num_mix_metadata_epochs=1,
    num_mix_metadata_warmup_epochs=1,
    maximum_mix_metadata_epochs=3,
    mix_metadata_name_1='cat',
    mix_metadata_name_2='lnk',
    mix_metadata_k=3,
)

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT021.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', tn_targ=1000, margin=0.3, 
                               tau=0.1, apply_softmax=True, n_negatives=10, m_lw=0.3, meta_prefix='cat', 
                               use_encoder_parallel=True, task_repr_type='tok', meta_repr_type='cls')
model.init_dr_head()

Some weights of DBT021 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight', 'encoder.meta_layer_norm.bias', 'encoder.meta_layer_norm.weight', 'encoder.meta_projector.bias', 'encoder.meta_projector.weight', 'encoder.meta_transform.bias', 'encoder.meta_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dset, valid_dset = block.train.dset.sample(n=100, seed=50), block.test.dset.sample(n=100, seed=50)

In [ ]:
train_dset, valid_dset = block.train.dset, block.test.dset

In [ ]:
metric = PrecRecl(block.n_lbl, valid_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl, 
                  pk=5, rk=5, rep_pk=[1, 3, 5], rep_rk=[5])

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    data_collator=block.collator, 
    train_dataset=train_dset, 
    eval_dataset=valid_dset,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
dl = learn.get_train_dataloader()

In [ ]:
batch = next(iter(dl))

In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 'cat2data_data2ptr', 'cat2lbl2data_idx', 'cat2lbl2data_input_ids', 'cat2lbl2data_attention_mask', 'cat2lbl2data_data2ptr', 'cat2lbl2data_lbl2data2ptr', 'data_input_ids', 'data_attention_mask'])

In [ ]:
batch['cat2data_data2ptr'].shape, batch['cat2data_idx'].shape

(torch.Size([1600]), torch.Size([7664]))

In [ ]:
batch['cat2data_data2ptr']

tensor([5, 5, 3,  ..., 2, 5, 1])

In [ ]:
learn.train_dataset.meta.cat_meta.data_meta = sparse.csr_matrix((693082, 656086), dtype=np.float32)
learn.train_dataset.meta.cat_meta._store_indices()

In [ ]:
batch = next(iter(dl))

In [ ]:
batch['cat2data_data2ptr'].shape, batch['cat2data_idx'].shape

(torch.Size([1600]), torch.Size([0]))

In [ ]:
batch['cat2data_data2ptr']

tensor([0, 0, 0,  ..., 0, 0, 0])

In [ ]:
learn.args.use_teacher_data_representation = False
learn.args.use_teacher_lbl_representation = False

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return learn.predict(valid_dset)
    

In [ ]:
o = func()

> /tmp/ipykernel_33850/2962927837.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return learn.predict(valid_dset)
      4 



ipdb>  b learn._build_lbl_index


Breakpoint 2 at /tmp/ipykernel_33850/1688291077.py:23


ipdb>  r


> /tmp/ipykernel_33850/1688291077.py(25)_build_lbl_index()
2    23 @patch
     24 def _build_lbl_index(self:XCLearner, dataset:Optional[Dataset]=None):
---> 25     dataset = dataset if self.eval_dataset is None else self.eval_dataset
     26     dataset = dataset if self.train_dataset is None else self.train_dataset
     27 



...

ipdb>  n


> /tmp/ipykernel_33850/2047086755.py(13)_get_lbl_representation()
     11                 dset = self._get_dataset(dataset, dset_type='data', use_metadata=self.args.use_centroid_data_metadata)
     12                 dataloader = self.get_test_dataloader(dset)
---> 13                 data_rep = self.get_representation(dataloader, representation_attribute=self.args.centroid_data_attribute_representation)
     14 
     15             lbl_data = self.train_dataset.data.data_lbl.T



ipdb>  n


  0%|          | 0/434 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
learn.train_dataset.meta.cat_meta.data_meta

<693082x656086 sparse matrix of type '<class 'numpy.float32'>'
	with 3390902 stored elements in Compressed Sparse Row format>

In [ ]:
learn.train_dataset.meta.cat_meta.curr_data_meta[10]

[53220, 86188, 174716, 174717]

In [ ]:
learn.args.prune_metadata_threshold=0.0
learn.args.prune_metadata_topk=3

learn.args.use_data_metadata_for_pruning = False
learn.args.output_representation_attribute = 'data_repr'

In [ ]:
learn.train_dataset.meta.cat_meta.data_meta

<693082x656086 sparse matrix of type '<class 'numpy.float32'>'
	with 3390902 stored elements in Compressed Sparse Row format>

In [ ]:
learn.train_dataset[10]['cat2data_input_text']

['Agglutinative languages',
 'Language families',
 'Austroasiatic languages',
 'Sino-Austronesian languages']

In [ ]:
learn.prune_metadata()

  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/411 [00:00<?, ?it/s]

  0%|          | 0/1656 [00:00<?, ?it/s]

  0%|          | 0/550 [00:00<?, ?it/s]

In [ ]:
learn.train_dataset[10]['cat2data_input_text']

['Agglutinative languages',
 'Austroasiatic languages',
 'Sino-Austronesian languages']

In [ ]:
from itertools import chain

In [ ]:
indices = list(chain(*learn.train_dataset.meta.cat_meta.curr_data_meta))

In [ ]:
len(indices)

1681856

In [ ]:
%%time
learn.get_augmentation_metadata()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/411 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1h 14min 33s, sys: 1min 15s, total: 1h 15min 48s
Wall time: 4min 10s


In [ ]:
def func():
    import pdb; pdb.set_trace()
    learn.prune_metadata()

In [ ]:
func()

In [ ]:
from IPython.core.debugger import Pdb; 

In [ ]:
def func():
    Pdb().set_trace()
    learn.train()
    

In [ ]:
func()

> /tmp/ipykernel_32410/2062820902.py(3)func()
      1 def func():
      2     Pdb().set_trace()
----> 3     learn.train()
      4 



ipdb>  c


Step,Training Loss,Validation Loss


ValueError: Model does not contain `m_teacher`.

In [ ]:
%debug

> /tmp/ipykernel_32410/3239534861.py(7)mix_metadata()
      5     else: pct = (self.args.num_mix_metadata_warmup_epochs - epochs_trained)/(self.args.maximum_mix_metadata_epochs - epochs_trained)
      6 
----> 7     if pct < 0: raise ValueError(f'Mixing percentage cannot be negative: {pct:.2f}')
      8     self.train_dataset.mix_meta_dataset(self.args.mix_metadata_name_1, self.args.mix_metadata_name_2, 
      9                                         pct=pct, k=self.args.mix_metadata_k)



ipdb>  epochs_trained


2


ipdb>  self.args.maximum_mix_metadata_epochs


3


ipdb>  (self.args.num_mix_metadata_warmup_epochs - epochs_trained)/(self.args.maximum_mix_metadata_epochs - epochs_trained)


-1.0


ipdb>  self.args.num_mix_metadata_warmup_epochs


1


ipdb>  q


In [ ]:
learn.train()

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
%%time
o = learn.predict(valid_dset)

> /tmp/ipykernel_17097/2037360939.py(61)predict()
     59         #debug
     60 
---> 61         self.gather_function, self._gen_kwargs = self.accelerator.gather, gen_kwargs
     62         self._memory_tracker.start()
     63 



ipdb>  n


> /tmp/ipykernel_17097/2037360939.py(62)predict()
     60 
     61         self.gather_function, self._gen_kwargs = self.accelerator.gather, gen_kwargs
---> 62         self._memory_tracker.start()
     63 
     64         test_dataloader = self.get_test_dataloader(test_dataset)



ipdb>  


> /tmp/ipykernel_17097/2037360939.py(64)predict()
     62         self._memory_tracker.start()
     63 
---> 64         test_dataloader = self.get_test_dataloader(test_dataset)
     65         start_time = time.time()
     66 



...

ipdb>  


> /scratch/scai/phd/aiz218323/Projects/xcai/xcai/models/radga.py(1075)forward()
   1073                 data_fused_repr = self.fused_dr(data_fused_embed, data_attention_mask)
   1074 
-> 1075         return EncoderOutput(
   1076             rep=data_repr,
   1077             fused_rep=data_fused_repr,



ipdb>  c


/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


CPU times: user 1h 49min 55s, sys: 1min 48s, total: 1h 51min 44s
Wall time: 26min 9s


In [ ]:
o.metrics

{'test_loss': 0.08889014273881912,
 'test_P@1': 0.17256569867335156,
 'test_P@3': 0.11264024636415412,
 'test_P@5': 0.08082922569928175,
 'test_N@1': 0.17256569862365723,
 'test_N@3': 0.17039525508880615,
 'test_N@5': 0.17370185256004333,
 'test_PSP@1': 0.16356071325613564,
 'test_PSP@3': 0.16702394838073903,
 'test_PSP@5': 0.16916310160335513,
 'test_PSN@1': 0.16356070339679718,
 'test_PSN@3': 0.17378103733062744,
 'test_PSN@5': 0.18096312880516052,
 'test_R@5': 0.18238680984822833,
 'test_runtime': 532.6447,
 'test_samples_per_second': 333.271,
 'test_steps_per_second': 0.208}